In [230]:
import pandas as pd

class Data:
    def __init__(self,selected_date,taxi,location,selected_day,yellow_taxi_data,green_taxi_data,hv_data,**kwargs) -> None:
        
        self.selected_date=selected_date
        self.selected_taxi=taxi
        self.selected_day=selected_day
        self.location=location
        
        #  data of csv
        self.y_taxi=yellow_taxi_data
        self.g_taxi=green_taxi_data
        self.hv=hv_data

        self.dicti=kwargs
        self.dates_list=[]
        self.previous_year_ride_counts={}


    def date_range(self):
        dates_list=[item.strftime("%Y/%m/%d") for item in  pd.date_range(pd.Timestamp(self.selected_date)-pd.offsets.Day(3),freq='D',periods=5)]
        future_dates=[item for item in pd.date_range(start=pd.Timestamp(self.selected_date)+pd.offsets.Day(2),freq='D',periods=2)]
        for item in future_dates:
            dates_list.append(item.strftime('%Y/%m/%d'))
        self.dates_list=dates_list
        return dates_list

    def previous_values(self)->dict:
        """if you want to show graphs of dates that are present in df"""
        previ=["2020","2021","2022"]
        for previous_year in previ:
            single_year_data=[]
            for single_date in self.dates_list:
                month1 = str(single_date[5:7])
                day1=str(single_date[8:])
                print(month1)
                x=previous_year+"-"+month1+"-"+day1
                dataframe1,condition_mask=None,None
                self.selected_taxi="Yellow Taxi"
                conditions={
                    "PickupBorough":str(self.location),
                    "RideType":str(self.selected_taxi),
                    "Date":str(x)
                }
               
        
#  1 cond
                if self.selected_taxi=="Green Taxi":
                    dataframe1=self.g_taxi
                    condition_mask = (
                        (dataframe1['PickupBorough'] == conditions['PickupBorough']) &
                        (dataframe1['RideType'] == conditions['RideType']) &
                        (dataframe1['Date'] == conditions['Date'])
                    )
#  2 cond

                elif self.selected_taxi=="Yellow Taxi":
                    dataframe1=self.y_taxi
                    condition_mask = (
                        (dataframe1['PickupBorough'] == conditions['PickupBorough']) &
                        (dataframe1['RideType'] == conditions['RideType']) &
                        (dataframe1['Date'] == conditions['Date'])
                    )

                else:
                    dataframe1=self.hv
                    conditions["RideType"]="FHVHV Taxi"
                    condition_mask = (
                        (dataframe1['PickupBorough'] == conditions['PickupBorough']) &
                        (dataframe1['RideType'] == conditions['RideType']) &
                        (dataframe1['Date'] == conditions['Date'])
                    )
                
                RC_for_previous_year = dataframe1.RideCount[condition_mask]
                print(int(RC_for_previous_year))
                single_year_data.append(int(RC_for_previous_year))
            self.previous_year_ride_counts[previous_year]=single_year_data

    def workday_con(self)->bool:
        """conditions for checking season and workday"""
        if self.selected_day  in ["Mon","Tue","Wed","Thu","Fri"]:
            workday=1
            return workday
        else:
            workday=0
            return workday
        
    def selected_season_con(self,month)->str:
        if month in [12, 1, 2]:
            season="Winter"
            return season
        elif month in [3, 4, 5]:
            season="Spring"
            return season
        elif month in [6, 7, 8]:
            season="Summer"
            return season
        elif month in [9, 10, 11]:
            season="Autumn"
            return season
        
    def checking_results(result:int)->int:
        if result < 0:
            result=0
        elif result >= 0:
            pass
        return result
    
    def api_parameters(self,single_date_to_predict)->dict:
        year1 = int(single_date_to_predict[:4])
        month1 = int(single_date_to_predict[5:7])
        day1=int(single_date_to_predict[9:])
        workday=self.workday_con()
        season=self.selected_season_con(month=self.selected_date.month)
        data =  {
        "Inputs": {
            "data": [
            {
                "PickupBorough": self.location, 
                "Year": year1, 
                "Month": month1,
                "Day": day1, 
                "Season": season, 
                "DayOfWeek": self.selected_day, 
                "Workday": workday
            }
            ]
        },
        "GlobalParameters": 1.0
        }
        return data
    
    def clean_result(self,api_result)->int:
        if api_result < 0:
            api_result=0
        elif api_result >= 0:
            pass
        return api_result
    
    def percentage(self,year_2022_values,predicted_values):
        percen_list = [round(((int(predict_value) - int(past_value)) / int(past_value)) * 100, 3) for past_value,predict_value in zip(year_2022_values,predicted_values)]
        return percen_list
    
    def create_df(self,results):
        data_2021=self.previous_year_ride_counts["2021"]
        data_2022=self.previous_year_ride_counts["2022"]
        
        # Convert dates to strings
        percentage_values=self.percentage(year_2022_values=data_2022,predicted_values=results)
        if len(self.dates_list) != len(results):
            return "Error: The 'dates' and 'results' lists must have the same length."
        else:
            try:
                chart_data = pd.DataFrame({
                    "Date": self.dates_list,
                    "Predicted_Values": results,
                    "2021": data_2021,
                    "2022": data_2022,
                    "Percentage":percentage_values
                })
            except:
                pass

        return chart_data


  

In [231]:


hv_data=pd.read_csv("../data/fhvhv.csv")
yellow_taxi_data=pd.read_csv("../data/yellow.csv")
green_taxi_data=pd.read_csv("../data/green.csv")
green_taxi_data.Date=green_taxi_data.Date.astype(str)
# print(green_taxi_data.head())
# print(yellow_taxi_data.head())

data=Data(selected_date="2021-01-01",selected_day="Sun",taxi="Yellow Taxi",
          location="Bronx",yellow_taxi_data=yellow_taxi_data,green_taxi_data=green_taxi_data,hv_data=hv_data)

          

date_list=data.date_range()   # date range to be return
data.previous_values()  # should give reault as the name

# print(data.previous_values())

      

12
558
12
600
12
658
01
437
01
361
01
357
01
270
12
69
12
80
12
75
01
333
01
301
01
262
01
551
12
145
12
122
12
111
01
64
01
46
01
66
01
78


In [ ]:
conditions = {
                "PickupBorough": "Manhattan",
                "RideType": "Yellow Taxi",
                "Date":f"{'1'}/{'01'}/{'2023'}"
                # "RideCount":""
            }
# print(self.location)
# print(self.selected_taxi)
# print(f"{day1}/{month1}/{previous_year}")
try:
    condition_mask = (yellow_taxi_data[list(conditions.keys())] == list(conditions.values())).all(axis=1)
except:
    print("operation failed")
print(condition_mask)
print(yellow_taxi_data[condition_mask])
            

In [192]:
import pandas as pd

# Assuming yellow_taxi_data is a DataFrame
yellow_taxi_data=pd.read_csv("../data/yellow.csv")
# Ensure the "Date" column is converted to string
yellow_taxi_data['Date'] = yellow_taxi_data['Date'].astype(str)
yellow_taxi_data.head()

# Define conditions
conditions = {
    "PickupBorough": "Manhattan",
    "RideType": "Green Taxi",
    "Date": "2022-01-01"  # Convert the date format to match the DataFrame
}
print(green_taxi_data.head())
dataframe1=green_taxi_data
try:
    # Create a boolean mask based on conditions
    condition_mask = (
        (dataframe1['PickupBorough'] == conditions['PickupBorough']) &
        (dataframe1['RideType'] == conditions['RideType']) &
        (dataframe1['Date'] == conditions['Date'])
    )
except Exception as e:
    print(f"Operation failed with exception: {e}")

dataframe1[condition_mask]


         Date PickupBorough    RideType  RideCount  Year  Month  Day  Season  \
0  2014-01-01      Brooklyn  Green Taxi       6725  2014      1    1  Winter   
1  2014-01-01        Queens  Green Taxi       8717  2014      1    1  Winter   
2  2014-01-01     Manhattan  Green Taxi       6700  2014      1    1  Winter   
3  2014-01-01         Bronx  Green Taxi       2879  2014      1    1  Winter   
4  2014-01-01           EWR  Green Taxi          0  2014      1    1  Winter   

  DayOfWeek  Workday  
0       Wed        1  
1       Wed        1  
2       Wed        1  
3       Wed        1  
4       Wed        1  


,Date,PickupBorough,RideType,RideCount,Year,Month,Day,Season,DayOfWeek,Workday
17536,2022-01-01,Manhattan,Green Taxi,570,2022,1,1,Winter,Sat,0


In [ ]:
def previous_values(self)->dict:
        """if you want to show graphs of dates that are present in df"""
        previ=["2020","2021","2022"]
        for previous_year in previ:
            single_year_data=[]
            for single_date in self.dates_list:
                month1 = str(single_date[5:7])
                day1=str(single_date[8:])
                conditions = {
                        "PickupBorough": self.location,
                        "RideType": self.selected_taxi,
                        "Date":f"{day1}/{month1}/{previous_year}"
                }
                condition_mask = (self.data[list(conditions.keys())] == list(conditions.values())).all(axis=1)
                
                RC_for_previous_year = (self.data.RideCount[condition_mask])
                
                single_year_data.append(int(RC_for_previous_year))
                
            self.previous_year_ride_counts[previous_year]=single_year_data

In [ ]:


hv_data=pd.read_csv("../data/fhvhv.csv")
yellow_taxi_data=pd.read_csv("../data/yellow.csv")
green_taxi_data=pd.read_csv("../data/green.csv")
green_taxi_data.Date=green_taxi_data.Date.astype(str)
# print(green_taxi_data.head())
# print(yellow_taxi_data.head())

data=Data(selected_date="2023-01-01",selected_day="Sun",taxi="Yellow Taxi",
          location="Manhattan",yellow_taxi_data=yellow_taxi_data,green_taxi_data=green_taxi_data,hv_data=hv_data)

          

date_list=data.date_range()   # date range to be return
data.previous_values()  # should give reault as the name

# print(data.previous_values())

      
# data.previous_values()  # should give reault as the name

for item in date_list:
  api_parameter=data.api_parameters(single_date_to_predict=item) 
      # api required parameters  
      
if taxi_type=="Green Taxi":
  api_result=api_call.url_call(url=green_taxi_api,data=api_parameter)
        
if taxi_type=="High Volume For-Hire Vehicle":
  api_result=api_call.url_call(url=hv_api,data=api_parameter)
        
if taxi_type=="Yellow Taxi":
  api_result=api_call.url_call(url=yellow_taxi_api,data=api_parameter)

       # give result to url call
      
  api_result=data.clean_result(api_result) # clean the result